In [14]:
from py2neo import Graph, Path
from py2neo import Node, Relationship
from pymongo import MongoClient
import json

config = json.load(open("config.json"))

graph = Graph("http://"+config["user"]+":"+config["password"]+"@"+config["server"]+":7474")

client = MongoClient()
client = MongoClient(config["server"], 27017)
db = client.Biomedical

## Merge crawling data into storage

In [12]:
import bson

with open('BME.bson','rb') as f:
    data = bson.decode_all(f.read())

In [13]:
data

[{'_id': ObjectId('5ae0252c5be253c92c73903b'),
  'faculty': 'http://bme.columbia.edu/people-by-type/faculty',
  'nodelink': 'http://bme.columbia.edu/',
  'nodename': 'bme',
  'research-9': 'http://bme.columbia.edu/research-9',
  'resources-2': 'http://bme.columbia.edu/resources-2'},
 {'_id': ObjectId('5ae0264f5be253c92c73903c'),
  'adjunct-faculty': 'http://bme.columbia.edu/people-by-type/adjunct-faculty',
  'affiliated-faculty': 'http://bme.columbia.edu/people-by-type/affiliated-faculty',
  'joint-faculty': 'http://bme.columbia.edu/people-by-type/joint-faculty',
  'nodelink': 'http://bme.columbia.edu/people-by-type/faculty',
  'nodename': 'faculty',
  'staff': 'http://bme.columbia.edu/people-by-type/staff'},
 {'Cardiac research': 'Cardiac biomechanics, cardiac tissue engineering and state-of-the-art in vivo cardiac imaging.',
  'Microscale engineering and nanotechnology:': 'Cell positioning and molecular design, MEMs and lab-on-a-chip diagnostic devices, micro- and nano-fabrication te

In [30]:
for d in data:
    institute = ["Columbia University"]
    if 'Email' in d:
        cat = 'Person'
    else:
        cat = 'Thing'
    if db.Node.find({'label':[d['nodename']],'category':cat}).count() > 0:
        db.Test.update_one({'label':[d['nodename']],'category':cat}, \
                           {"$set": {'Email':[d['Email']],'abstract':[d['introduction0']],\
                                    'Publications':d['Publications'],'Institute':institute}}, upsert=False)
    else:
        if cat == 'Person':
            if 'Publications' in d:
                db.Test.insert_one({'label':[d['nodename']],'category':cat,\
                           'Email':[d['Email']],'abstract':[d['introduction0']],\
                           'Publications':d['Publications'],'Institute':institute})
            else:
                #for some unstructed publications, add into publication
                features = ['nodename','_id','nodelink','Email','introduction0','introduction1','Publications']
                pub = []
                for p in d:
                    if p not in features:
                        pub.append(p+d[p])
                if 'introduction0' not in d:
                    intro = ['Person in '+institute[0]]
                else:
                    intro = [d['introduction0']]
                db.Test.insert_one({'label':[d['nodename']],'category':cat,\
                           'Email':[d['Email']],'abstract':intro,\
                           'Publications':pub,'Institute':institute})

## Merge into knowledge graph

In [31]:
from py2neo import Graph, Path
from py2neo import Node, Relationship
from pymongo import MongoClient
import json

config = json.load(open("config.json"))

graph = Graph("http://"+config["user"]+":"+config["password"]+"@"+config["server"]+":7474")

client = MongoClient()
client = MongoClient(config["server"], 27017)
db = client.Biomedical

In [38]:
graphnode = Node('University',title = ['Columbia University'],content = 'Columbia University')
graph.create(graphnode)

In [39]:
insti = 'Columbia University'
for t in db.Test.find({}):
    query = 'MATCH (c) WHERE c.title=["' + t['label'][0] + '"] RETURN c'
    if len(graph.run(query).data()) == 0:
        abstract = ['This is the '+t['category']+'of node '+insti]
        gn = Node(t['category'],title = [t['label'][0]],content = abstract)
        graph.create(gn)
    else:
        gn = graph.run(query).data()
    createR = 'MATCH (p:'+t['category']+' {title:["'+t['label'][0]+'"]}),'+\
                    '(c:University {title:["'+insti+'"]})' +'CREATE (c)-[:`workplace`]->(p)'
    print(createR)
    graph.run(createR)

MATCH (p:Thing {title:["bme"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["faculty"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["research-9"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["resources-2"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["faculty"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["joint-faculty"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["adjunct-faculty"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["affiliated-faculty"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Thing {title:["staff"]}),(c:University {title:["Columbia Un

MATCH (p:Person {title:["henry-spotnitz"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)
MATCH (p:Person {title:["stephen-h-tsang"]}),(c:University {title:["Columbia University"]})CREATE (c)-[:`workplace`]->(p)


In [44]:
#Add publication relationsihps
for t in db.Test.find({}):
    if t['category'] == 'Person':
        for pub in t['Publications']:
            pub = pub.replace('"',"<")
            query = 'MATCH (c) WHERE c.title=["' + [pub] + '"] RETURN c'
            if len(graph.run(query).data()) == 0:
                abstract = ['This is the publications of person '+t['label'][0]]
                gn = Node('Publications',title = [pub],content = abstract)
                graph.create(gn)

            createR = 'MATCH (p:'+'Publications'+' {title:["'+pub+'"]}),'+\
                            '(c:Person {title:["'+t['label'][0]+'"]})' +'CREATE (c)-[:`publicated`]->(p)'
            #print(createR)
            graph.run(createR)

MATCH (p:Publications {title:["CVhttp://bme.columbia.edu/files/seasdepts/biomedical-engineering/cv-hess.pdf"]}),(c:Person {title:["henry-hess"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["Selected PublicationsH. Hess and J.L. Ross: “Non-Equilibrium Assembly of Microtubules: From Molecules to Autonomous Chemical Robots”, Chemical Society Reviews 46, 5570-5587 (2017)
Y. Zhang, S. Tsitkov, H. Hess: ”Proximity Does Not Contribute to Activity Enhancement in the Glucose Oxidase-Horseradish Peroxidase Cascade”, Nature Communications 7:13982 (2016)
E.L.P. Dumont, C. Do and H. Hess: “Molecular wear of microtubules propelled by surface-adhered kinesins”, Nature Nanotechnology 10, 166-169 (2015)
O. Idan and H. Hess: “Diffusive transport phenomena in artificial enzyme cascades on scaffolds”, Nature Nanotechnology, 7, 769-770 (2012)
H. Hess: <Engineeringapplicationsofbiomolecularmotors<, Annual Review of Biomedical Engineering, 13, 429-450 (2011)
T. Fischer, A. Agarwal and H. He

MATCH (p:Publications {title:["l"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["o"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["a"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["d"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["M"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["a"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["Y"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[","]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["S"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

MATCH (p:Publications {title:["d"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["e"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["-"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["f"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["i"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["e"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["l"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["d"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["o"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

MATCH (p:Publications {title:["v"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["e"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["l"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["a"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["p"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["p"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["r"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["o"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["a"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

MATCH (p:Publications {title:["E"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["M"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["C"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[","]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["<"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["N"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["e"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["u"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["r"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

...

MATCH (p:Publications {title:[")"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[","]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[" "]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["1"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["5"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["6"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["7"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["0"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["-"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

MATCH (p:Publications {title:[" "]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["m"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["o"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["l"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["e"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["c"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["u"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["l"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["a"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

MATCH (p:Publications {title:["n"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["g"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[" "]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["o"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["f"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[" "]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["t"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["h"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["e"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

MATCH (p:Publications {title:["e"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["s"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["s"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[" "]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["2"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["0"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["0"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:["9"]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated`]->(p)
MATCH (p:Publications {title:[","]}),(c:Person {title:["elizabeth-mc-hillman"]})CREATE (c)-[:`publicated

KeyboardInterrupt: 